In [53]:
import os, datetime
import pandas as pd
from sdv.metadata import Metadata
from sdv.single_table import CTGANSynthesizer, GaussianCopulaSynthesizer, CopulaGANSynthesizer
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.single_table import CopulaGANSynthesizer
from rdt.transformers.categorical import OrderedUniformEncoder
from rdt.transformers.datetime import OptimizedTimestampEncoder
from rdt.transformers.numerical import GaussianNormalizer
from sdmetrics.visualization import get_column_plot
from sdv.sampling import Condition
import numpy as np

pd.set_option('display.max_columns', None)

## Data Loading

In [7]:
# File path to the Excel file
file_path = 'Dataset_2.0_Akkodis.xlsx'

# Import the dataset into a pandas DataFrame
original_data = pd.read_excel(file_path)

In [8]:
original_data.head()

,ID,Candidate State,Age Range,Residence,Sex,Protected category,TAG,Study area,Study Title,Years Experience,...,Akkodis headquarters,Current Ral,Expected Ral,Technical Skills,Standing/Position,Comunication,Maturity,Dynamism,Mobility,English
0,71470,Hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],...,Modena,22-24 K,24-26 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,71470,Hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],...,Modena,22-24 K,24-26 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,71470,Hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],...,Modena,22-24 K,24-26 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,71470,Hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],...,Modena,22-24 K,24-26 K,2.0,2.0,1.0,2.0,2.0,3.0,3.0
4,71470,Hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],...,Modena,22-24 K,24-26 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
original_data.shape

(21377, 39)

In [10]:
original_data.head()

,ID,Candidate State,Age Range,Residence,Sex,Protected category,TAG,Study area,Study Title,Years Experience,...,Akkodis headquarters,Current Ral,Expected Ral,Technical Skills,Standing/Position,Comunication,Maturity,Dynamism,Mobility,English
0,71470,Hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],...,Modena,22-24 K,24-26 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,71470,Hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],...,Modena,22-24 K,24-26 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,71470,Hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],...,Modena,22-24 K,24-26 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,71470,Hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],...,Modena,22-24 K,24-26 K,2.0,2.0,1.0,2.0,2.0,3.0,3.0
4,71470,Hired,31 - 35 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOSAR, CAN, C, C++, MATLAB/SIMULINK, VECTOR/...",Automation/Mechatronics Engineering,Five-year degree,[1-3],...,Modena,22-24 K,24-26 K,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
original_data.describe(include='all')

,ID,Candidate State,Age Range,Residence,Sex,Protected category,TAG,Study area,Study Title,Years Experience,...,Akkodis headquarters,Current Ral,Expected Ral,Technical Skills,Standing/Position,Comunication,Maturity,Dynamism,Mobility,English
count,21377.000000,21377,21377,21374,21377,85,10647,21332,21377,21377,...,2120,4156,4119,5955.000000,5974.000000,5968.00000,5964.000000,5965.000000,5974.000000,5944.000000
unique,NaN,7,7,2507,2,2,2245,48,7,7,...,18,18,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,Imported,26 - 30 years,TURIN » Turin ~ Piedmont,Male,Article 1,.,Mechanical engineering,Five-year degree,[0],...,Turin,Not available,Not available,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,7515,10080,3491,16722,76,633,3866,12522,12376,...,557,1401,1650,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,40519.752023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.177666,2.265149,2.32004,2.285714,2.296060,2.225979,2.764973
std,23615.657709,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.620178,0.573715,0.59193,0.585352,0.588163,0.810831,0.540574
min,15.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,1.000000,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000
25%,20228.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.000000,2.000000,2.00000,2.000000,2.000000,2.000000,3.000000
50%,40413.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,2.000000,2.000000,2.00000,2.000000,2.000000,2.000000,3.000000
75%,61043.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.000000,3.000000,3.00000,3.000000,3.000000,3.000000,3.000000


In [12]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21377 entries, 0 to 21376
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        21377 non-null  int64  
 1    Candidate State          21377 non-null  object 
 2    Age Range                21377 non-null  object 
 3    Residence                21374 non-null  object 
 4    Sex                      21377 non-null  object 
 5    Protected category       85 non-null     object 
 6    TAG                      10647 non-null  object 
 7    Study area               21332 non-null  object 
 8    Study Title              21377 non-null  object 
 9    Years Experience         21377 non-null  object 
 10   Sector                   12214 non-null  object 
 11   Last Role                12214 non-null  object 
 12   Year of insertion        21377 non-null  object 
 13   Year of Recruitment      2389 non-null   object 
 14   Recru

In [13]:
original_data.nunique()

ID                          12263
 Candidate State                7
 Age Range                      7
 Residence                   2507
 Sex                            2
 Protected category             2
 TAG                         2245
 Study area                    48
 Study Title                    7
 Years Experience               7
 Sector                        14
 Last Role                   2698
 Year of insertion              6
 Year of Recruitment            7
 Recruitment Request          359
 Assumption Headquarters       18
 Job Family Hiring              7
 Job Title Hiring              19
 event_type__val               13
 event_feedback                15
 linked_search__key          1746
 Overall                        8
 Job Description              350
 Candidate Profile            350
 Years Experience.1             7
 Minimum Ral                   15
 Ral Maximum                   18
 Study Level                    7
 Study Area.1                  26
 Akkodis headq

In [ ]:
'''for column in data.columns:
    print(f'{column}: {data[column].unique()}')
    print('Unique values: ', data[column].unique())
    print('\n')'''

## Data cleaning

Opzione Tancre: 
 - tenere tutte le colonne e togliere le righe duplicate facendo un check con gli IDs.
 - eventualmente ordinare gli IDs in ordine crescente e poi volendo fare un reset degli indici.

In [17]:
original_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21377 entries, 0 to 21376
Data columns (total 39 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        21377 non-null  int64  
 1    Candidate State          21377 non-null  object 
 2    Age Range                21377 non-null  object 
 3    Residence                21374 non-null  object 
 4    Sex                      21377 non-null  object 
 5    Protected category       85 non-null     object 
 6    TAG                      10647 non-null  object 
 7    Study area               21332 non-null  object 
 8    Study Title              21377 non-null  object 
 9    Years Experience         21377 non-null  object 
 10   Sector                   12214 non-null  object 
 11   Last Role                12214 non-null  object 
 12   Year of insertion        21377 non-null  object 
 13   Year of Recruitment      2389 non-null   object 
 14   Recru

In [30]:
for i in sorted(original_data[' Last Role'].unique(), key=str):
    print(i)
    

 Corse Powertrain Division
 Drone Contest
 Engineer
 Financial Advisor
 Internship and Thesis at University
 OPERATIONS ENGINEER
 Project Manager
 Researcher
 Simulation Manager for Galileo satellites & ground Operations
 System Engineer
 researchers & Data Scientists
-
.
.NET Developer
.NET Full Stack Developer
.NET Programmer Analyst
.NET developer
.NET programmer
/
3D Mechanical Designer
3D PIPING ENGINEER OIL, GAS & REFINING SECTOR
3rd level metalworker employee 
????
ADAS Algorithm Engineer
ADAS Developer
ADAS Engineer
ADAS Passive Safety
ADAS System Resident Engineer
ADAS Validation Engineer
ADAS development internal
ADMINISTRATIVE EMPLOYEE
ADMINISTRATIVE EMPLOYEE - PURCHASING OFFICE
ADMINISTRATIVE EMPLOYEE / GRAPHIC DESIGNER
AERODYNAMICS ENGINEER
AEROSPACE FUNCTIONAL TESTER
AI Software Engineer
AI mobile software engineer
AI&C System Completion Coordinator
AIRWORTHINESS ENGINEER
AIT Consultant
AIT/AIV Engineer
AO NPO Lavockina-Secretariat and Report
ASSOCIATE PROJECT MANAGER
AUT

In [43]:
data = original_data.copy()

def organize_data(data):
    data.columns = data.columns.str.strip()
    data = data.map(lambda x: x.strip() if isinstance(x, str) else x)

    # Drop duplicate rows
    data = data.drop_duplicates()

    # Drop the tilde in the 'Overall' column
    data['Overall'] = data['Overall'].str.lstrip('~ ')

    '''# Extract 'City', 'Province' and 'Region' from the column 'Residence'
    # ??
    data[['City', 'Province', 'Region']] = data['Residence'].str.extract(r'^(.*?) » (.*?) ~ (.*)$')
    data = data.drop(columns=['Residence']) '''

    # Convert the columns 'Year of insertion' and 'Year of Recruitment' to integers
    data['Year of insertion'] = pd.to_numeric(data['Year of insertion'].str.strip('[]'), errors='coerce').astype('Int64')
    data['Year of Recruitment'] = pd.to_numeric(data['Year of Recruitment'].str.strip('[]'), errors='coerce').astype('Int64')
    
    undesired_values = ['????', '-', '.', '/']
    data.loc[data['Last Role'].isin(undesired_values), 'Last Role'] = np.nan

    # Group the same IDs in a unique row
    def group_ids(df):
        # Count non-NaN values in each row
        df['non_nan_count'] = df.notna().sum(axis=1)
        # Keep the row with the highest non-NaN count per ID
        df = df.loc[df.groupby('ID')['non_nan_count'].idxmax()] 
        # Drop helper column
        df = df.drop(columns=['non_nan_count'])
        return df
    
    data = group_ids(data)

    return data

data = organize_data(data)

In [44]:
data.shape

(12263, 39)

In [41]:
'''for i in sorted(data['Last Role'].unique(), key=str):
    print(i)'''

"for i in sorted(data['Last Role'].unique(), key=str):\n    print(i)"

In [65]:
metadata = Metadata.detect_from_dataframe(data=data)

# Set column Candidate State to categorical
metadata.update_column(
    column_name='Candidate State',
    sdtype='categorical')

In [66]:
metadata.validate()

In [67]:
synthesizer = GaussianCopulaSynthesizer(
    metadata,
    locales='it_IT',
#    verbose=True
)

synthesizer.auto_assign_transformers(data)

synthesizer.fit(data)

synthetic_data = synthesizer.sample(num_rows=1000)

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sdv/single_table/base.py:104: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(


In [72]:
synthesizer = CTGANSynthesizer(
    metadata,
    locales='it_IT',
    epochs=5
#    verbose=True
)

synthesizer.auto_assign_transformers(data)

synthesizer.fit(data)

synthetic_data = synthesizer.sample(num_rows=1000)

PerformanceAlert: Using the CTGANSynthesizer on this data is not recommended. To model this data, CTGAN will generate a large number of columns.

Original Column Name    Est # of Columns (CTGAN)
Candidate State         7
Age Range               7
Residence               2420
Sex                     2
Protected category      3
TAG                     2201
Study area              49
Study Title             7
Years Experience        7
Sector                  15
Year of insertion       6
Year of Recruitment     8
Recruitment Request     354
Assumption Headquarters 19
Job Family Hiring       8
Job Title Hiring        20
event_type__val         13
event_feedback          16
linked_search__key      1354
Overall                 5
Job Description         339
Candidate Profile       340
Years Experience.1      8
Minimum Ral             15
Ral Maximum             19
Study Level             8
Study Area.1            26
Akkodis headquarters    19
Current Ral             19
Expected Ral            1

In [73]:
synthesizer.get_transformers()


{'ID': AnonymizedFaker(function_name='random_int', function_kwargs={'min': 0, 'max': 16777216}, cardinality_rule='unique'),
 'Candidate State': None,
 'Age Range': None,
 'Residence': None,
 'Sex': None,
 'Protected category': None,
 'TAG': None,
 'Study area': None,
 'Study Title': None,
 'Years Experience': None,
 'Sector': None,
 'Last Role': AnonymizedFaker(function_name='bothify', function_kwargs={'text': 'sdv-pii-?????', 'letters': '0123456789abcdefghijklmnopqrstuvwxyz'}),
 'Year of insertion': None,
 'Year of Recruitment': None,
 'Recruitment Request': None,
 'Assumption Headquarters': None,
 'Job Family Hiring': None,
 'Job Title Hiring': None,
 'event_type__val': None,
 'event_feedback': None,
 'linked_search__key': None,
 'Overall': None,
 'Job Description': None,
 'Candidate Profile': None,
 'Years Experience.1': None,
 'Minimum Ral': None,
 'Ral Maximum': None,
 'Study Level': None,
 'Study Area.1': None,
 'Akkodis headquarters': None,
 'Current Ral': None,
 'Expected Ral':

In [74]:
synthetic_data.head(20)

,ID,Candidate State,Age Range,Residence,Sex,Protected category,TAG,Study area,Study Title,Years Experience,Sector,Last Role,Year of insertion,Year of Recruitment,Recruitment Request,Assumption Headquarters,Job Family Hiring,Job Title Hiring,event_type__val,event_feedback,linked_search__key,Overall,Job Description,Candidate Profile,Years Experience.1,Minimum Ral,Ral Maximum,Study Level,Study Area.1,Akkodis headquarters,Current Ral,Expected Ral,Technical Skills,Standing/Position,Comunication,Maturity,Dynamism,Mobility,English
0,8179297,Imported,> 45 years,TURIN » Turin ~ Piedmont,Male,NaN,NaN,Legal,Five-year degree,[0],Others,sdv-pii-zgg4a,2019,2023,RS18.0351 - Junior Recruiter,NaN,NaN,NaN,BM interview,OK,RS22.0970,3 - High,NaN,NaN,[0],NaN,NaN,NaN,Aeronautical/Aerospace/Astronautics Engineering,Poggibonsi,NaN,NaN,2.0,NaN,NaN,2.0,NaN,NaN,3.0
1,7414570,Hired,26 - 30 years,ROME » Rome ~ Lazio,Male,NaN,NaN,Mechanical engineering,Five-year degree,[3-5],Oil and Gas,sdv-pii-p25pg,2023,<NA>,NaN,Pisa,NaN,Support,BM interview,OK,RS22.0066,1 - Low,system microprocessor development op. Real tim...,NaN,NaN,24-26K,NaN,NaN,NaN,Valenzano,36-38 K,34-36 K,NaN,2.0,2.0,NaN,3.0,1.0,2.0
2,390390,In selection,26 - 30 years,ALBENGA » Savona ~ Liguria,Male,NaN,NaN,Energy and Nuclear Engineering,Five-year degree,[0],Consulting,sdv-pii-jbyyg,2021,<NA>,NaN,Turin,NaN,NaN,HR interview,KO (manager),NaN,4 - Top,NaN,previous experience in green energy project is...,[+10],24-26K,28-30K,Three-year degree,NaN,NaN,NaN,NaN,3.0,3.0,3.0,3.0,NaN,3.0,3.0
3,10658106,Imported,> 45 years,BISCEGLIE » Barletta-Andria-Trani ~ Puglia,Male,NaN,NaN,Management Engineering,Five-year degree,[+10],Aeronautics,NaN,2022,<NA>,NaN,NaN,Engineering,NaN,BM interview,OK,NaN,NaN,NaN,NaN,[+10],NaN,20-22K,Three-year degree,NaN,NaN,NaN,Not available,NaN,2.0,2.0,NaN,NaN,3.0,3.0
4,10110043,Hired,> 45 years,TURIN » Turin ~ Piedmont,Male,NaN,"-, ANSYS FLUEN, OPENFORM, PLAXIS 2D, CFD,",Other scientific subjects,Five-year degree,[+10],NaN,sdv-pii-wos1l,2021,2022,NaN,NaN,NaN,Consultant,NaN,OK,RS21.1201,2 - Medium,NaN,NaN,NaN,22-24K,NaN,NaN,Other,NaN,NaN,34-36 K,2.0,2.0,NaN,NaN,3.0,2.0,NaN
5,13862948,In selection,36 - 40 years,CHIERI » Turin ~ Piedmont,Female,NaN,"JAVA BACKEND, CLOUD",Mechanical engineering,Three-year degree,[0],Automotive,sdv-pii-k3liu,2023,<NA>,Quality assurance Crio propulsion,Rome,NaN,Senior Consultant,Technical interview,OK,RS19.0501,4 - Top,NaN,NaN,NaN,NaN,- 20K,NaN,NaN,NaN,NaN,NaN,1.0,2.0,NaN,2.0,3.0,NaN,NaN
6,8301953,Hired,26 - 30 years,ROME » Rome ~ Lazio,Male,NaN,"AEROSPACE ENGINEERING, 3D CAD DESIGN, SOLID ED...",Economic - Statistics,Professional qualification,[0],Automotive,sdv-pii-pz2ly,2023,<NA>,NaN,Modena,NaN,NaN,BM interview,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40-42K,NaN,Scientific maturity,NaN,Not available,NaN,2.0,NaN,3.0,2.0,NaN,NaN,NaN
7,15234470,First contact,26 - 30 years,TURIN » Turin ~ Piedmont,Male,NaN,"AUTOMOTIVE, INCA, CANALYZER, MDA, MATLAB, SIMU...",computer engineering,Five-year degree,[0],Aeronautics,sdv-pii-3ipy2,2022,<NA>,NaN,NaN,NaN,NaN,Technical interview,NaN,NaN,3 - High,NaN,NaN,NaN,22-24K,NaN,NaN,NaN,NaN,24-26 K,NaN,3.0,NaN,2.0,2.0,NaN,3.0,3.0
8,6162191,Imported,36 - 40 years,CLOSED » Siena ~ Tuscany,Female,NaN,"C#, J2EE, JAA ENTERPRSE, PHP, JAVA, JAVA, BACK...",Management Engineering,Five-year degree,[+10],Others,sdv-pii-b2xi1,2019,2023,NaN,Pisa,NaN,Junior Consultant,CV request,OK (live),NaN,NaN,NaN,Senior DHSE engineer if possible with previous...,NaN,NaN,28-30K,NaN,Other scientific subjects,NaN,26-28 K,30-32 K,NaN,NaN,3.0,4.0,2.0,4.0,1.0
9,4334613,Hired,> 45 years,BAUNEI » Ogliastra ~ Sardinia,Male,NaN,"ELECTRICAL ENGINEER, FIRMWARE, HARDWARE",electronic Engineering,Five-year degree,[1-3],Machining - Heavy Industry,NaN,2018,2022,Search: Configuration Manager,NaN,NaN,NaN,BM interview,KO (language skills),RS22.1054,3 - High,NaN,SW tester with skills in test scripting and cr...,NaN,NaN,28-30K,NaN,NaN,NaN,NaN,40-42 K,2.0,2.0,3.0,2.0,3.0,2.0,NaN
